In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from PIL import Image, ImageFile
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import re
import emoji
import time
from torchvision import transforms
from glob import glob
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import pickle
import torch
import open_clip
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import torchvision.transforms as transforms
from transformers import CLIPProcessor, CLIPModel
import pickle
import os
import torch
import numpy as np
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
ImageFile.LOAD_TRUNCATED_IMAGES = True
import GPUtil
import psutil

In [ ]:
gpus = GPUtil.getGPUs()
for gpu in gpus:
    print(f"\nGPU {gpu.id}:")
    print(f"  Name: {gpu.name}")
    print(f"  Total Memory: {gpu.memoryTotal} MB")
    print(f"  Memory Used: {gpu.memoryUsed} MB")
    print(f"  Memory Free: {gpu.memoryFree} MB")
    print(f"  Temperature: {gpu.temperature} C")

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
cpu_percent = psutil.cpu_percent(interval=1) 
print(f"CPU Usage: {cpu_percent}%")
ram = psutil.virtual_memory()
print(f"Total RAM: {ram.total / (1024 ** 3):.2f} GB")
print(f"Used RAM: {ram.used / (1024 ** 3):.2f} GB")
print(f"Available RAM: {ram.available / (1024 ** 3):.2f} GB")
print(f"RAM Usage: {ram.percent}%")


In [ ]:
clip_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor()            
])

def get_all_image_paths(directory):
    image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}  
    image_paths = []

    for root, _, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                image_paths.append(os.path.join(root, file))

    return image_paths

def process_images(image_paths, batch_size=64):
    all_embeddings_image = []
    all_filenames = []

    for i in tqdm(range(0, len(image_paths), batch_size), desc="Processing Images"):
        batch_paths = image_paths[i:i + batch_size]
        batch_images = []
        for img_path in batch_paths:
            try:
                image = Image.open(img_path).convert("RGB")  
                batch_images.append(image)  
                all_filenames.append(os.path.basename(img_path))  
            except Exception as e:
                print(f"Error processing {img_path}: {e}")

        if batch_images:
            with torch.no_grad():  
                inputs = processor(images=batch_images, return_tensors="pt").to(device)
                embeddings = model.get_image_features(pixel_values=inputs.pixel_values)  
            all_embeddings_image.append(embeddings.cpu().numpy())  

    return np.vstack(all_embeddings_image), all_filenames


In [ ]:
image_dir = ""
image_paths = get_all_image_paths(image_dir)
print(f"Found {len(image_paths)} images.")

In [ ]:
main_directory = image_dir
subdirectories = os.listdir(main_directory)  
pickle_file_location = ''

In [ ]:
for path in tqdm(subdirectories, desc="Processing User IDs", unit="user"):
    print(f"User_ID : {path}")
    start = time.time()
    image_folder = os.path.join(main_directory, path) 
    image_paths = glob(os.path.join(image_folder, "*.jpg"))
    valid_image_paths = [img_path for img_path in image_paths if os.path.getsize(img_path) > 0]
    try:
        features = process_images(valid_image_paths)
        dirc = Path(pickle_file_location + path)
        dirc.mkdir(parents=True, exist_ok=True)
        pd.concat([pd.DataFrame(features[1]).rename(columns = {0:'Tweet_ID'}),pd.DataFrame(features[0])], axis = 1).to_pickle(pickle_file_location + path + "/Image_Embedding_" + path + ".pkl")
        end = time.time()
        print(f"Total time taken: {end - start:.2f} seconds")
    except:
        pass

In [ ]:
def load_pickles_from_directory(root_dir):
    all_data = []
    for subdir, _, files in os.walk(root_dir):
        user_id = os.path.basename(subdir)  
        for file in files:
            if file.endswith('.pkl'):
                file_path = os.path.join(subdir, file)
                try:
                    with open(file_path, 'rb') as f:
                        data = pickle.load(f)
                        if isinstance(data, pd.DataFrame):
                            data['User_ID'] = user_id 
                            if 'Tweet_ID' in data.columns:
                                data['Tweet_ID'] = data['Tweet_ID'].str.replace('.jpg', '', regex=False)
                            all_data.append(data)
                        else:
                            print(f"Skipping {file_path}: Not a DataFrame")
                except Exception as e:
                    print(f"Error loading {file_path}: {e}")   
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame() 

In [ ]:
root_directory = ""  
merged_df = load_pickles_from_directory(root_directory)
print(merged_df.head())


In [ ]:
merged_df.to_pickle('')

In [ ]:
merged_df

# FAISS

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
import time
import pandas as pd
import faiss
import numpy as np
import networkx as nx
import sys
import GPUtil
import psutil

In [ ]:
print(faiss.__version__)

In [ ]:
merged_df

In [ ]:
merged_df[['Tweet_ID', 'User_ID']].reset_index().rename(columns = {'index': 'Text_Index'}).to_csv('Politix_Image_Index_Tweet_ID_User_ID_Mapping_final.csv', index = False)

In [ ]:
df_index = merged_df[['Tweet_ID', 'User_ID']].reset_index().rename(columns = {'index': 'Text_Index'})

In [ ]:
f_arr = np.array(merged_df.iloc[:,1:-1])
f_arr = f_arr.astype("float32") 
f_arr = np.ascontiguousarray(f_arr)  
faiss.normalize_L2(f_arr)  
dim = f_arr.shape[1]  
cpu_index = faiss.IndexFlatIP(dim)  
cpu_index.add(f_arr)

In [ ]:
threshold=0.95

In [ ]:
limits, distances, indices = cpu_index.range_search(f_arr, threshold)

In [ ]:
similar_pairs_dict = {}
seen_pairs = set()  

for i in range(len(limits) - 1):
    start, end = limits[i], limits[i + 1]
    
    for j in range(start, end):
        a, b = i, indices[j]
        if a != b: 
            pair = (min(a, b), max(a, b))
            if pair not in seen_pairs:
                seen_pairs.add(pair)
                if a not in similar_pairs_dict:
                    similar_pairs_dict[a] = []
                similar_pairs_dict[a].append({"neighbor": b, "distance": distances[j]})

df_res = pd.DataFrame([
    {"source": key, "neighbor": entry["neighbor"], "distance": entry["distance"]}
    for key, values in similar_pairs_dict.items() for entry in values
])


In [ ]:
df_res.to_pickle('')

In [ ]:
df_res.loc[df_res.distance>= 0.99].sample(20)

In [ ]:
df_res.loc[(df_res.distance<= 0.99) & (df_res.distance>= 0.98)].sample(5)